<a href="https://colab.research.google.com/github/Rin-Sanity/test1/blob/main/application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **対話エージェント作成**

In [ ]:
!apt install mecab libmecab-dev mecab-ipadic-utf8


Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-14build9 [306 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-utils amd64 0.996-14build9 [4,850 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-ipadic all 2.7.0-20070801+main-3 [6,718 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mecab amd64 0.996-14build9 [136 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy/main amd64 mec

In [ ]:
!pip install mecab-python3
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=254f84b260b323eab28d6033564ff90f5dcd90bf54cca6d32817ab10bc19fb23
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [ ]:
import MeCab
from os.path import dirname, join, normpath

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tagger = MeCab.Tagger()

print(tagger.parse('私は私のことが好きなあなたが好きです'))


私	ワタクシ	ワタクシ	私-代名詞	代名詞			0
は	ワ	ハ	は	助詞-係助詞			
私	ワタクシ	ワタクシ	私-代名詞	代名詞			0
の	ノ	ノ	の	助詞-格助詞			
こと	コト	コト	事	名詞-普通名詞-一般			2
が	ガ	ガ	が	助詞-格助詞			
好き	スキ	スキ	好き	形状詞-一般			2
な	ナ	ダ	だ	助動詞	助動詞-ダ	連体形-一般	
あなた	アナタ	アナタ	貴方	代名詞			2
が	ガ	ガ	が	助詞-格助詞			
好き	スキ	スキ	好き	形状詞-一般			2
です	デス	デス	です	助動詞	助動詞-デス	終止形-一般	
EOS



In [ ]:


tagger = MeCab.Tagger()


def tokenize(text):
    node = tagger.parseToNode(text)

    tokens = []
    while node:
        if node.surface != '':
            tokens.append(node.surface)

        node = node.next

    return tokens

In [ ]:
tokenize('私は私のことが好きなあなたが好きです')

['私', 'は', '私', 'の', 'こと', 'が', '好き', 'な', 'あなた', 'が', '好き', 'です']

In [ ]:


tagger = MeCab.Tagger('-Owakati')

print(tagger.parse('私は私のことが好きなあなたが好きです'))

assert tagger.parse('私は私のことが好きなあなたが好きです') == '私 は 私 の こと が 好き な あなた が 好き です \n'


私 は 私 の こと が 好き な あなた が 好き です 



In [ ]:


tagger = MeCab.Tagger('-Owakati')


def tokenize(text):
    return tagger.parse(text).strip().split(' ')


### **リスト12**

In [ ]:



def calc_bow(tokenized_texts):  # <2>
    # Build vocabulary <3>
    vocabulary = {}
    for tokenized_text in tokenized_texts:
        for token in tokenized_text:
            if token not in vocabulary:
                vocabulary[token] = len(vocabulary)

    n_vocab = len(vocabulary)

    # Build BoW Feature Vector <4>
    bow = [[0] * n_vocab for i in range(len(tokenized_texts))]
    for i, tokenized_text in enumerate(tokenized_texts):
        for token in tokenized_text:
            index = vocabulary[token]
            bow[i][index] += 1

    return vocabulary, bow


# 入力文のlist
texts = [
    '私は私のことが好きなあなたが好きです',
    '私はラーメンが好きです',
    '富士山は日本一高い山です',
]

tokenized_texts = [tokenize(text) for text in texts]
vocabulary, bow = calc_bow(tokenized_texts)


In [ ]:
vocabulary

{'私': 0,
 'は': 1,
 'の': 2,
 'こと': 3,
 'が': 4,
 '好き': 5,
 'な': 6,
 'あなた': 7,
 'です': 8,
 'ラーメン': 9,
 '富士': 10,
 '山': 11,
 '日本': 12,
 '一': 13,
 '高い': 14}

In [ ]:
bow

[[2, 1, 1, 1, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 1, 1, 1]]

### **リスト13**

In [ ]:
from collections import Counter




def calc_bow(tokenized_texts):
    counts = [Counter(tokenized_text)
              for tokenized_text in tokenized_texts]  # <1>
    sum_counts = sum(counts, Counter())  # <2>
    vocabulary = sum_counts.keys()

    bow = [[count[word] for word in vocabulary]
           for count in counts]  # <3>

    return bow


# 入力文のlist
texts = [
    '私は私のことが好きなあなたが好きです',
    '私はラーメンが好きです',
    '富士山は日本一高い山です',
]

tokenized_texts = [tokenize(text) for text in texts]
bow = calc_bow(tokenized_texts)


In [ ]:
Counter(['A','A','A','B','B','B','C',])

Counter({'A': 3, 'B': 3, 'C': 1})

### **リスト14**

In [ ]:

texts = [
    '私は私のことが好きなあなたが好きです',
    '私はラーメンが好きです。',
    '富士山は日本一高い山です',
]

# Bag of Words計算
vectorizer = CountVectorizer(tokenizer=tokenize)  # <2>
vectorizer.fit(texts)  # <3>
bow = vectorizer.transform(texts)  # <4>


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
bow

<3x16 sparse matrix of type '<class 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [ ]:
bow.__class__

scipy.sparse._csr.csr_matrix

In [ ]:
print(bow)

  (0, 1)	1
  (0, 2)	2
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 10)	2
  (0, 14)	2
  (1, 0)	1
  (1, 2)	1
  (1, 4)	1
  (1, 7)	1
  (1, 8)	1
  (1, 10)	1
  (1, 14)	1
  (2, 4)	1
  (2, 7)	1
  (2, 9)	1
  (2, 11)	1
  (2, 12)	2
  (2, 13)	1
  (2, 15)	1


In [ ]:
bow.toarray()

array([[0, 1, 2, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 2, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 2, 1, 0, 1]])

### **リスト16**

In [ ]:
import pandas as pd
from os.path import join
from sklearn.feature_extraction.text import CountVectorizer

# CSVファイルのパスを直接指定
csv_path = './training_data.csv'  # 相対パスまたは絶対パスを使用

# CSVファイルの読み込み
training_data = pd.read_csv(csv_path)
training_texts = training_data['text']

# Bag of Words計算
vectorizer = CountVectorizer(tokenizer=tokenize)
vectorizer.fit(training_texts)
bow = vectorizer.transform(training_texts)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### **リスト17**

In [ ]:
import MeCab
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

class DialogueAgent:
    def __init__(self):
        self.tagger = MeCab.Tagger()

    def _tokenize(self, text):
        node = self.tagger.parseToNode(text)
        tokens = []
        while node:
            if node.surface != '':
                tokens.append(node.surface)
            node = node.next
        return tokens

    def train(self, texts, labels):
        vectorizer = CountVectorizer(tokenizer=self._tokenize)
        bow = vectorizer.fit_transform(texts)

        classifier = SVC()
        classifier.fit(bow, labels)

        self.vectorizer = vectorizer
        self.classifier = classifier

    def predict(self, texts):
        bow = self.vectorizer.transform(texts)
        return self.classifier.predict(bow)

# 以下のコードはJupyter NotebookやGoogle Colabで実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
replies_path = './replies.csv'  # 応答データのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

with open(replies_path) as f:
    replies = f.read().split('\n')

input_text = '名前を教えてよ'
predictions = dialogue_agent.predict([input_text])
predicted_class_id = predictions[0]

print(replies[predicted_class_id])

while True:
    input_text = input()
    predictions = dialogue_agent.predict([input_text])
    predicted_class_id = predictions[0]

    print(replies[predicted_class_id])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


私は〇〇といいます
あ
こんにちは


KeyboardInterrupt: ignored

### **リスト19**

In [ ]:
import MeCab
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

class DialogueAgent:
    def __init__(self):
        self.tagger = MeCab.Tagger()

    def _tokenize(self, text):
        node = self.tagger.parseToNode(text)
        tokens = []
        while node:
            if node.surface != '':
                tokens.append(node.surface)
            node = node.next
        return tokens

    def train(self, texts, labels):
        pipeline = Pipeline([
            ('vectorizer', CountVectorizer(tokenizer=self._tokenize)),
            ('classifier', SVC()),
        ])
        pipeline.fit(texts, labels)
        self.pipeline = pipeline

    def predict(self, texts):
        return self.pipeline.predict(texts)

# 以下のコードはJupyter NotebookやGoogle Colabで実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
replies_path = './replies.csv'  # 応答データのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

with open(replies_path) as f:
    replies = f.read().split('\n')

input_text = '名前を教えてよ'
predictions = dialogue_agent.predict([input_text])
predicted_class_id = predictions[0]

print(replies[predicted_class_id])
while True:
    input_text = input()
    predictions = dialogue_agent.predict([input_text])
    predicted_class_id = predictions[0]

    print(replies[predicted_class_id])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


私は〇〇といいます
あ
こんにちは


KeyboardInterrupt: ignored

### **リスト21**

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score


# 以下のコードはJupyter NotebookやGoogle Colabで実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
test_data_path = './test_data.csv'  # テストデータのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

# テストデータの読み込み
test_data = pd.read_csv(test_data_path)

predictions = dialogue_agent.predict(test_data['text'])

# 精度の計算
print(accuracy_score(test_data['label'], predictions))


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.574468085106383


# **前処理**

In [ ]:
!pip install neologdn
import neologdn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.2-cp310-cp310-linux_x86_64.whl size=219148 sha256=cba004b0c4373a160344af046b6b39fb405a99f2330081c5f834a55cb2e4f6a4
  Stored in directory: /root/.cache/pip/wheels/bd/87/af/2a36d74f666a8428943b70d71c5e9dd740435bf671f210672c
Successfully built neologdn


In [ ]:
print(neologdn.normalize('「初めてのTensorFlow」は定価2200円+税です'))
print(neologdn.normalize('「初めての　ＴｅｎｓｏｒＦｌｏｗ」は定価２２００円＋税です'))
print(neologdn.normalize('｢初めての TensorFlow｣は定価2200円＋税です'))

「初めてのTensorFlow」は定価2200円+税です
「初めてのTensorFlow」は定価2200円+税です
「初めてのTensorFlow」は定価2200円+税です


In [ ]:
#小文字化、大文字化
text='「初めてのTensorFlow」は定価2200円+税です'
print(text.lower())
print(text.upper())

「初めてのtensorflow」は定価2200円+税です
「初めてのTENSORFLOW」は定価2200円+税です


In [63]:
import unicodedata

normalized = unicodedata.normalize('NFKC', '㈱リックテレコム')

assert normalized == '(株)リックテレコム'
print(normalized)


(株)リックテレコム


In [65]:
import unicodedata

from sklearn.feature_extraction.text import CountVectorizer



def normalize_and_tokenize(text):
    normalized = unicodedata.normalize('NFKC', text)
    return tokenize(normalized)


texts = [
    'ディスプレイを買った',
    'ディスプレイを買った',
]

vectorizer = CountVectorizer(tokenizer=normalize_and_tokenize)
vectorizer.fit(texts)

print('bow:')
print(vectorizer.transform(texts).toarray())

print('vocabulary:')
print(vectorizer.vocabulary_)


bow:
[[1 1 1 1]
 [1 1 1 1]]
vocabulary:
{'ディスプレイ': 2, 'を': 1, '買っ': 3, 'た': 0}


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### **リスト4**

In [89]:
import MeCab

tagger = MeCab.Tagger()


def tokenize(text):
    node = tagger.parseToNode(text)
    result = []
    while node:
        features = node.feature.split(',')  # <1>

        if features[0] != 'BOS/EOS':  # <2>
            token = features[7] if features[7] != '*' else node.surface  # <1>
            result.append(token)

        node = node.next

    return result


print(tokenize('本を読んだ'))
print(tokenize('本を読みました'))


['本', 'を', '読む', 'た']
['本', 'を', '読む', 'ます', 'た']


### **リスト5**

In [77]:
import MeCab

tagger = MeCab.Tagger()


def tokenize(text, stop_words):  # <1>
    node = tagger.parseToNode(text)
    result = []
    while node:
        features = node.feature.split(',')

        if features[0] != 'BOS/EOS':
            token = features[7] if features[7] != '*' else node.surface
            if token not in stop_words:  # <2>
                result.append(token)

        node = node.next

    return result


stop_words = ['て', 'に', 'を', 'は', 'です', 'ます']  # <3>

print(tokenize('本を読んだ', stop_words))
print(tokenize('本を読みました', stop_words))


['本', '読む', 'た']
['本', '読む', 'た']


In [78]:


tagger = MeCab.Tagger()


def tokenize(text):
    node = tagger.parseToNode(text)
    result = []
    while node:
        features = node.feature.split(',')

        if features[0] != 'BOS/EOS':
            if features[0] not in ['助詞', '助動詞']:  # <1>
                token = features[7] if features[7] != '*' else node.surface
                result.append(token)

        node = node.next

    return result


print(tokenize('本を読んだ'))
print(tokenize('本を読みました'))


['本', '読む']
['本', '読む']


### **リスト8**

In [79]:
import re


def tokenize_numbers(text):
    return re.sub(r'\d+', ' SOMENUMBER ', text)


print(tokenize_numbers('卵を1個買ったよ！'))
print(tokenize_numbers('卵を2個買ったよ！'))
print(tokenize_numbers('卵を10個買ったよ！'))


卵を SOMENUMBER 個買ったよ！
卵を SOMENUMBER 個買ったよ！
卵を SOMENUMBER 個買ったよ！


### **リスト9**

In [86]:
import MeCab
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import unicodedata
import neologdn

class DialogueAgent:
    def __init__(self):
        self.tagger = MeCab.Tagger()

    def _tokenize(self, text):
        text = unicodedata.normalize('NFKC', text)
        text = neologdn.normalize(text)
        text = text.lower()

        node = self.tagger.parseToNode(text)
        result = []
        while node:
            features = node.feature.split(',')

            if features[0] != 'BOS/EOS':
                if features[0] not in ['助詞', '助動詞']:
                    token = features[5] if features[5] != '*' else node.surface
                    result.append(token)

            node = node.next

        return result

    def train(self, texts, labels):
        vectorizer = CountVectorizer(tokenizer=self._tokenize)
        bow = vectorizer.fit_transform(texts)

        classifier = SVC()
        classifier.fit(bow, labels)

        self.vectorizer = vectorizer
        self.classifier = classifier

    def predict(self, texts):
        bow = self.vectorizer.transform(texts)
        return self.classifier.predict(bow)


# 以下のコードは Jupyter Notebook や Google Colab で実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
replies_path = './replies.csv'  # 応答データのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

with open(replies_path) as f:
    replies = f.read().split('\n')

input_text = '名前は？'
predictions = dialogue_agent.predict([input_text])
predicted_class_id = predictions[0]

print(replies[predicted_class_id])

while True:
    input_text = input()
    predictions = dialogue_agent.predict([input_text])
    predicted_class_id = predictions[0]

    print(replies[predicted_class_id])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


私は〇〇といいます
う
こんにちは
おはよう
こんにちは
おやすみ
こんにちは
今何時「
こんにちは
子守唄
こんにちは
スポーツしたい
子守唄でも歌いましょうか？


KeyboardInterrupt: ignored

In [91]:
"""
isort:skip_file
"""
from os.path import normpath, dirname, join
import MeCab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import pandas as pd
import unicodedata
import neologdn


class DialogueAgent:
    def __init__(self):
        self.tagger = MeCab.Tagger()

    def _tokenize(self, text):
        text = unicodedata.normalize('NFKC', text)  # <1>
        text = neologdn.normalize(text)  # <2>
        text = text.lower()  # <3>

        node = self.tagger.parseToNode(text)
        result = []
        while node:
            features = node.feature.split(',')

            if features[0] != 'BOS/EOS':
                if features[0] not in ['助詞', '助動詞']:  # <4>
                    token = features[7] \
                            if features[5] != '*' \
                            else node.surface  # <5>
                    result.append(token)

            node = node.next

        return result

    def train(self, texts, labels):
        vectorizer = CountVectorizer(tokenizer=self._tokenize)
        bow = vectorizer.fit_transform(texts)  # <2>

        classifier = SVC()
        classifier.fit(bow, labels)

        # <3>
        self.vectorizer = vectorizer
        self.classifier = classifier

    def predict(self, texts):
        bow = self.vectorizer.transform(texts)
        return self.classifier.predict(bow)


# 以下のコードはJupyter NotebookやGoogle Colabで実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
replies_path = './replies.csv'  # 応答データのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

with open(replies_path) as f:
    replies = f.read().split('\n')

    input_text = '名前は？'
    predictions = dialogue_agent.predict([input_text])
    predicted_class_id = predictions[0]

    print(replies[predicted_class_id])

    while True:
        input_text = input()
        predictions = dialogue_agent.predict([input_text])
        predicted_class_id = predictions[0]

        print(replies[predicted_class_id])



/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


私は〇〇といいます
スポーツ
こんにちは
スポーツしたい
良かったですね！
お昼食べたい
こんにちは
名前は
私は〇〇といいます
何が好き
何か一緒に食べましょう。
好きなものは
えっ、ありがとうございます
プレゼントあげます
えっ、ありがとうございます
これどうぞ
こんにちは


KeyboardInterrupt: ignored

In [92]:
import pandas as pd
from sklearn.metrics import accuracy_score


# 以下のコードはJupyter NotebookやGoogle Colabで実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
test_data_path = './test_data.csv'  # テストデータのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

# テストデータの読み込み
test_data = pd.read_csv(test_data_path)

predictions = dialogue_agent.predict(test_data['text'])

# 精度の計算
print(accuracy_score(test_data['label'], predictions))


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.6702127659574468


# **形態素解析とわかち描き**

### **リスト1**

In [93]:
from os.path import dirname, join, normpath

import MeCab
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

MECAB_DIC_DIR = '/usr/lib/mecab/dic/mecab-ipadic-neologd'


class DialogueAgent:
    def __init__(self):
        self.tagger = MeCab.Tagger('-d {}'.format(MECAB_DIC_DIR))

    def _tokenize(self, text):
        node = self.tagger.parseToNode(text)

        tokens = []
        while node:
            if node.surface != '':
                tokens.append(node.surface)

            node = node.next

        return tokens

    def train(self, texts, labels):
        vectorizer = CountVectorizer(tokenizer=self._tokenize)
        bow = vectorizer.fit_transform(texts)  # <2>

        classifier = SVC()
        classifier.fit(bow, labels)

        # <3>
        self.vectorizer = vectorizer
        self.classifier = classifier

    def predict(self, texts):
        bow = self.vectorizer.transform(texts)
        return self.classifier.predict(bow)


# 以下のコードはJupyter NotebookやGoogle Colabで実行する場合
# ファイルのパスは直接指定します
# 以下のパスは例です。実際のファイルの場所に合わせて変更してください。

training_data_path = './training_data.csv'  # トレーニングデータのパス
replies_path = './replies.csv'  # 応答データのパス

# トレーニングデータの読み込み
training_data = pd.read_csv(training_data_path)

dialogue_agent = DialogueAgent()
dialogue_agent.train(training_data['text'], training_data['label'])

with open(replies_path) as f:
    replies = f.read().split('\n')

input_text = '名前を教えてよ'
predictions = dialogue_agent.predict([input_text])
predicted_class_id = predictions[0]

print(replies[predicted_class_id])
while True:
    input_text = input()
    predictions = dialogue_agent.predict([input_text])
    predicted_class_id = predictions[0]

    print(replies[predicted_class_id])


RuntimeError: ignored